# Manipulation de ressources dans Vortex (bis)

Dans le premier notebook/présentation, la façon dont Vortex fut "chargé" était occultée. Ici, soyons plus exlicites : puisque la présentation utilise les notebooks Jupyter et donc IPython, nous utilisons l'extension IPython développée spécialement pour Vortex.

Cette extension se nomme *ivortex* et se trouve dans le sous-répertoire ``project`` de la distribution de Vortex. Elle étend les commandes habituelles du shell IPython (la commande "magique" ``%vortex tmpcocoon`` permet de charger Vortex et de se placer dans un répertoire temporaire qui sera détruit en fin d'exécution).

In [1]:
%load_ext ivortex
%vortex tmpcocoon

# [2019/01/17-11:39:50][vortex.sessions][_set_rundir:0152][WARNING]: Session <root> is changing its working directory </sauvegarde/stage-vortex/presentation/nb>
# [2019/01/17-11:39:50][vortex.sessions][_set_rundir:0155][INFO]: Session <root> set rundir </home/meunierlf/vortex-workdir/auto_cocoon_dvgz79wa>


Vortex 1.6.0 loaded ( Thursday 17. January 2019, at 11:39:49 )
The working directory is now: /home/meunierlf/vortex-workdir/auto_cocoon_dvgz79wa/root


'/home/meunierlf/vortex-workdir/auto_cocoon_dvgz79wa'

Note: Un peu plus détail est donné dans la documentation. Voir: [Notebooks Summary > Working with Vortex in IPython and Notebooks](http://intra.cnrm.meteo.fr/algopy/sphinx/vortex/current/notebooks/00_ipython_and_notebooks_howto.html).

## Exemple précédement utilisé...

In [2]:
rhandlers = toolbox.input(# Input function options
                          verbose=False,
                          # Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term=3, geometry='euroc25',
                          nativefmt='grib', model='arpege', 
                          cutoff='production', date='2017060118',
                          origin='historic',
                          # Provider
                          vapp='arpege', vconf='pearp', member=2,
                          experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='my_pearp_gribfile', format='grib')
print(rhandlers)
rhandlers[0].clear()  # Nettoyage

True

On comprend désormais mieux ce qui se passe : le package **footprints** est utilisé pour créer les objets *resource*, *provider* et *container*.

## Utilisation de valeurs pré-définies

La plupart des scripts Vortex définissent des valeurs pré-définies (que ce soient les scripts Olive ou opérationels). Faisons la même chose ici :

In [3]:
# Le module toolbox fournit un raccourci vers le setup de footprints
import bronx
toolbox.defaults(
    date = bronx.stdtypes.date.Date('2017060118'),
    cutoff = 'production',
    model = 'arpege',
    geometry = vortex.data.geometries.get(tag='euroc25'),
)
# On peut également préciser un vapp/vconf par défaut
t.glove.vapp = 'arpege'
t.glove.vconf = 'pearp'  # En l'état de nos connaissances, je ne peux expliquer cela...


On en profite pour paramétrer correctement le module **toolbox**

In [4]:
toolbox.active_now = True
toolbox.active_verbose = False

Cela permet de grandement simplifier la définition du _resource's Handler_ :

In [6]:
rhandlers = toolbox.input(# Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term=3, origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='my_pearp_gribfile', format='grib')
print(rhandlers)
rhandlers[0].clear()  # Nettoyage

# [2019/01/17-11:40:03][vortex.data.handlers][_get_proxy:0682][WARNING]: The resource is already here: That should not happen at this stage !
# [2019/01/17-11:40:03][vortex.data.handlers][_get_proxy:0682][WARNING]: The resource is already here: That should not happen at this stage !
# [2019/01/17-11:40:03][vortex.data.stores][inarchiveget:1199][INFO]: inarchiveget on vortex://vsop.archive.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: my_pearp_gribfile)
# [2019/01/17-11:40:03][vortex.tools.storage][_ftpretrieve:0690][INFO]: ftpget on ftp://hendrix.meteo.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: my_pearp_gribfile)
# [2019/01/17-11:40:04][vortex.tools.net][get:0266][INFO]: FTP <get:/home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib>


True

## Le mécanisme de substitution de footprints (en pratique)

Souvent, certains attributs sont construits à partir d'autres... Par exemple, on aimerait que le nom du fichier GRIB local dépende du domaine, de l'échéance, du numéro de membre, ...

Rien de plus simple avec le mécanisme de substitution de footprints.

In [7]:
rhandlers = toolbox.input(# Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term=3, origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='GRIB_[geometry]_[member]+[term]', format='grib')
%ls
rhandlers[0].clear()  # Nettoyage

# [2019/01/17-11:40:07][vortex.data.stores][inarchiveget:1199][INFO]: inarchiveget on vortex://vsop.archive.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_<vortex.data.geometries.LonlatGeometry | tag='euroc25' id='Large european BDAP target' area='EUROC25' r='02dg500'>_002+03:00)
# [2019/01/17-11:40:07][vortex.tools.storage][_ftpretrieve:0690][INFO]: ftpget on ftp://hendrix.meteo.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_<vortex.data.geometries.LonlatGeometry | tag='euroc25' id='Large european BDAP target' area='EUROC25' r='02dg500'>_002+03:00)
# [2019/01/17-11:40:08][vortex.tools.net][get:0266][INFO]: FTP <get:/home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib>


'GRIB_<vortex.data.geometries.LonlatGeometry | tag='\''euroc25'\'' id='\''Large european BDAP target'\'' area='\''EUROC25'\'' r='\''02dg500'\''>_002+03:00'


True

* Le mécanisme de substitution est déclenché, dans des chaînes de caractères, par la syntaxe ``[quelquechose]``


Le résultat est très mitigé...

* Le format utilisé pour la géométrie ne convient pas du tout.
* Nous n'avons pas choisi le formatage pour le membre et l'échéance

### Utilisation du type des attributs

* La géométrie est un objet ayant un type bien défini dans Vortex. On peut donc utiliser ses attributs pour arriver à un nom plus satisfaisant :



In [8]:
geo = vortex.data.geometries.Geometry('euroc25')
print("Type d'une géométrie:", type(geo))  # Cela dépend de la géométries; globales, latlon, ...
print("La zone couverte par la geometrie (au sens de la BDAP):", geo.area)

Type d'une géométrie: <class 'vortex.data.geometries.LonlatGeometry'>
La zone couverte par la geometrie (au sens de la BDAP): EUROC25


* L'attribut *term* est de type **bronx.stdtypes.date.Time**, on aimerait utiliser ses attributs et *properties* pour formater l'échéance.

In [9]:
rhandlers = toolbox.input(# Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term=3, origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='GRIB_[geometry:area]_[member]+[term:fmthm]', format='grib')
%ls
rhandlers[0].clear()  # Nettoyage

# [2019/01/17-11:40:13][vortex.data.stores][inarchiveget:1199][INFO]: inarchiveget on vortex://vsop.archive.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_EUROC25_002+0003:00)
# [2019/01/17-11:40:13][vortex.tools.storage][_ftpretrieve:0690][INFO]: ftpget on ftp://hendrix.meteo.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_EUROC25_002+0003:00)
# [2019/01/17-11:40:14][vortex.tools.net][get:0266][INFO]: FTP <get:/home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib>


GRIB_EUROC25_002+0003:00


True

### Formatage des entiers

Avec des entiers (ou réels), on peut utiliser les codes de formatage classiques :

In [10]:
rhandlers = toolbox.input(# Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term=3, origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='GRIB_[geometry:area]_[member%04d]+[term:fmthm]', format='grib')
%ls
rhandlers[0].clear()  # Nettoyage

# [2019/01/17-11:40:17][vortex.data.stores][inarchiveget:1199][INFO]: inarchiveget on vortex://vsop.archive.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_EUROC25_0002+0003:00)
# [2019/01/17-11:40:17][vortex.tools.storage][_ftpretrieve:0690][INFO]: ftpget on ftp://hendrix.meteo.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_EUROC25_0002+0003:00)
# [2019/01/17-11:40:19][vortex.tools.net][get:0266][INFO]: FTP <get:/home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib>


GRIB_EUROC25_0002+0003:00


True

### Utilisation de méthodes plus complexes

On peut par exemple demander à un objet **Date** de faire des calculs. Ici on ajoute l'échéance à la date, afin d'obtenir la date de validité.

In [11]:
rhandlers = toolbox.input(# Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term=3, origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='GRIB_[geometry:area]_[date:addterm_ymdh]_[member%04d]',
                          format='grib')
%ls
rhandlers[0].clear()  # Nettoyage

# [2019/01/17-11:40:22][vortex.data.stores][inarchiveget:1199][INFO]: inarchiveget on vortex://vsop.archive.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_EUROC25_2017060121_0002)
# [2019/01/17-11:40:22][vortex.tools.storage][_ftpretrieve:0690][INFO]: ftpget on ftp://hendrix.meteo.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_EUROC25_2017060121_0002)
# [2019/01/17-11:40:22][vortex.tools.net][get:0266][INFO]: FTP <get:/home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib>


GRIB_EUROC25_2017060121_0002


True

## Le mécanisme d'expansion de footprints (en pratique)

Jusque là nous n'avons récupéré qu'un seul fichier à la fois. Cela pourrait vite devenir fastidieux pour les tâches nécessitant de nombreuses données d'entrée.

Le mécanisme d'expansion permet, avant la résolution des empreintes, de créer un grand nombre de descriptions de ressources à partir d'une description concise.

### Expansion de listes

(indication: voir l'attribut *term*)

In [12]:
rhandlers = toolbox.input(loglevel='warning',  # Reduction du niveau de logs
                          # Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term=[0,3,6], origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='GRIB_[geometry:area]_[member%04d]+[term:fmthm]', format='grib')
%ls
for rhandler in rhandlers:
    rhandler.clear()  # Nettoyage

GRIB_EUROC25_0002+0000:00  GRIB_EUROC25_0002+0003:00  GRIB_EUROC25_0002+0006:00


### Un principe finalement assez simple...

L'expansion est réalisée par une unique fonction de **footprints** : **footprints.util.expand**. Celle-ci reçoit un dictionnaire en entrée et renvoie une liste de dictionaires. Avec l'exemple précédent, cela donne :

In [13]:
input_dict = dict(kind='gridpoint', term=[0,3,6], origin='historic', nativefmt='grib',
                  member=2, experiment='DBLE', namespace='vortex.archive.fr', block='forecast',
                  local='GRIB_[geometry:area]_[member%04d]+[term:fmthm]', format='grib')
for i, a_dict in enumerate(fp.util.expand(input_dict)):
    print(">>>Dictionaire #{:d}".format(i))
    print(a_dict)

>>>Dictionaire #0
{'experiment': 'DBLE', 'local': 'GRIB_[geometry:area]_[member%04d]+[term:fmthm]', 'namespace': 'vortex.archive.fr', 'block': 'forecast', 'format': 'grib', 'origin': 'historic', 'member': 2, 'kind': 'gridpoint', 'nativefmt': 'grib', 'term': 0}
>>>Dictionaire #1
{'experiment': 'DBLE', 'local': 'GRIB_[geometry:area]_[member%04d]+[term:fmthm]', 'namespace': 'vortex.archive.fr', 'block': 'forecast', 'format': 'grib', 'origin': 'historic', 'member': 2, 'kind': 'gridpoint', 'nativefmt': 'grib', 'term': 3}
>>>Dictionaire #2
{'experiment': 'DBLE', 'local': 'GRIB_[geometry:area]_[member%04d]+[term:fmthm]', 'namespace': 'vortex.archive.fr', 'block': 'forecast', 'format': 'grib', 'origin': 'historic', 'member': 2, 'kind': 'gridpoint', 'nativefmt': 'grib', 'term': 6}


Utiliser le mécanisme d'extension est donc totalement équivalent à créer manuellement N sections via les commandes de la **toolbox**...

### Autre exemple: Liste sous forme d'une chaîne

(indication: voir l'attribut *term*)

In [14]:
rhandlers = toolbox.input(loglevel='warning',  # Reduction du niveau de logs
                          # Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term='0,3,6', origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='GRIB_[geometry:area]_[member%04d]+[term:fmthm]', format='grib')
%ls
for rhandler in rhandlers:
    rhandler.clear()  # Nettoyage

GRIB_EUROC25_0002+0000:00  GRIB_EUROC25_0002+0003:00  GRIB_EUROC25_0002+0006:00


### Autre exemple: Rangex

(indication: voir l'attribut *term*)

In [15]:
rhandlers = toolbox.input(loglevel='warning',  # Reduction du niveau de logs
                          # Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term=fp.util.rangex('0-12-3'), origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='GRIB_[geometry:area]_[member%04d]+[term:fmthm]', format='grib')
%ls
for rhandler in rhandlers:
    rhandler.clear()  # Nettoyage

GRIB_EUROC25_0002+0000:00  GRIB_EUROC25_0002+0006:00  GRIB_EUROC25_0002+0012:00
GRIB_EUROC25_0002+0003:00  GRIB_EUROC25_0002+0009:00


Autres exemples d'utilisation de *rangex* :

In [17]:
print(fp.util.rangex(0, 12, 3))
print(fp.util.rangex('0-12-3', shift=24))

[0, 3, 6, 9, 12]
[24, 27, 30, 33, 36]


Avec des heures/minutes :

In [18]:
print(fp.util.rangex('0:00', '3:00', '0:30'))

['0000:00', '0000:30', '0001:00', '0001:30', '0002:00', '0002:30', '0003:00']


Avec des chaînes plus complexes...

In [19]:
print(fp.util.rangex('0-12-3,18-36-6,48'))

[0, 3, 6, 9, 12, 18, 24, 30, 36, 48]


### Globbing

Il s'agit de scruter le répertoire courant pour construire des variables en analysant les noms de certains fichiers ...

In [20]:
for tfile in ('f_euroc25_0001:00', 'f_euroc25_0002:00', 'f_glob25_0002:00', 'f_a_trap'):
    sh.touch(tfile)

In [21]:
rhandlers = toolbox.output(# Method/Section
                           now=False, role='GridpointData',
                           # Resource
                           geometry='[glob:g]', term='[glob:t]',
                           kind='gridpoint', origin='historic', nativefmt='grib',
                           # Provider
                           member=2, experiment='ABCD', namespace='vortex.archive.fr',
                           block='forecast',
                           # Container
                           local='f_{glob:g:\w+}_{glob:t:\d+:\d+}', format='grib')
for rhandler in rhandlers:
    print('From:', rhandler.container.localpath())
    print('To:  ', rhandler.locate())

From: f_euroc25_0001:00
To:   meunierlf@hendrix.meteo.fr:/home/m/marp/marp999/vortex/arpege/pearp/A/B/C/D/20170601T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0001:00.grib
From: f_euroc25_0002:00
To:   meunierlf@hendrix.meteo.fr:/home/m/marp/marp999/vortex/arpege/pearp/A/B/C/D/20170601T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0002:00.grib
From: f_glob25_0002:00
To:   meunierlf@hendrix.meteo.fr:/home/m/marp/marp999/vortex/arpege/pearp/A/B/C/D/20170601T1800P/mb002/forecast/grid.arpege-forecast.glob25+0002:00.grib


In [22]:
sh.rmall('f_*')

## Petit exemple représentatif

In [23]:
rhandlers = toolbox.input(# Method/ection
                          loglevel='warning', role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          geometry='euroc25,glob05', term=fp.util.rangex('0-12-3'),
                          kind='gridpoint', origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='GRIB_[geometry:area]_[member%04d]+[term:fmthm]', format='grib')
%ls
for rhandler in rhandlers:
    rhandler.clear()  # Nettoyage

GRIB_EUROC25_0002+0000:00  GRIB_EUROC25_0002+0012:00  GRIB_GLOB05_0002+0009:00
GRIB_EUROC25_0002+0003:00  GRIB_GLOB05_0002+0000:00   GRIB_GLOB05_0002+0012:00
GRIB_EUROC25_0002+0006:00  GRIB_GLOB05_0002+0003:00
GRIB_EUROC25_0002+0009:00  GRIB_GLOB05_0002+0006:00



Questions éventuelles : *vortex.support@meteo.fr*